# Creación del modelo

In [1]:
import pandas as pd
import pickle
from scipy.stats import poisson


In [6]:
from google.colab import files
uploaded = files.upload()

Saving dict_table.pkl to dict_table.pkl


In [7]:
dict_table = pickle.load(open('dict_table.pkl','rb'))
df_data_historica = pd.read_csv('Clean_Mundiales_FIFA_registro_historico_limpio.csv')
df_fixture = pd.read_csv('Clean_Mundiales_FIFA_fixture_limpio.csv')

In [8]:
dict_table['Grupo A']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Mexico (H),0,0,0,0,0,0,0,0
1,2,South Africa,0,0,0,0,0,0,0,0
2,3,South Korea,0,0,0,0,0,0,0,0
3,4,UEFA Path D winner[a],0,0,0,0,0,0,0,0


In [9]:
df_data_historica

,Local,Visitante,Año,Goles_Locales,Goles_Visitantes,Goles_Totales
0,France,Mexico,1930,4,1,5
1,Argentina,France,1930,1,0,1
2,Chile,Mexico,1930,3,0,3
3,Chile,France,1930,1,0,1
4,Argentina,Mexico,1930,6,3,9
...,...,...,...,...,...,...
923,England,France,2022,1,2,3
924,Argentina,Croatia,2022,3,0,3
925,France,Morocco,2022,2,0,2
926,Croatia,Morocco,2022,2,1,3


In [10]:
df_fixture

,Local,Resultado,Visitante,Año
0,South Africa,Match 1,South Africa,2026
1,UEFA Path D winner,Match 2,UEFA Path D winner,2026
2,South Africa,Match 25,South Africa,2026
3,South Korea,Match 28,South Korea,2026
4,Mexico,Match 53,Mexico,2026
...,...,...,...,...
99,Winner Match 96,Match 100,Winner Match 96,2026
100,Winner Match 98,Match 101,Winner Match 98,2026
101,Winner Match 100,Match 102,Winner Match 100,2026
102,Loser Match 102,Match 103,Loser Match 102,2026


## Fuerza del equipo

In [12]:
# se divide el df en local y visitante
df_home = df_data_historica[['Local', 'Goles_Locales', 'Goles_Visitantes']]
df_away = df_data_historica[['Visitante', 'Goles_Locales', 'Goles_Visitantes']]


In [23]:
df_home.rename(columns={'Local':'equipo',
                        'Goles_Locales':'Goles_anotados',
                        'Goles_Visitantes': 'Goles_recibidos'}, inplace=True)
df_home

/tmp/ipython-input-2198903432.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_home.rename(columns={'Local':'equipo',


,equipo,Goles_anotados,Goles_recibidos
0,France,4,1
1,Argentina,1,0
2,Chile,3,0
3,Chile,1,0
4,Argentina,6,3
...,...,...,...
923,England,1,2
924,Argentina,3,0
925,France,2,0
926,Croatia,2,1


In [24]:
df_away.rename(columns={'Visitante':'equipo',
                        'Goles_Locales':'Goles_recibidos',
                        'Goles_Visitantes': 'Goles_anotados'}, inplace=True)
df_away

/tmp/ipython-input-2987974571.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_away.rename(columns={'Visitante':'equipo',


,equipo,Goles_recibidos,Goles_anotados
0,Mexico,4,1
1,France,1,0
2,Mexico,3,0
3,France,1,0
4,Mexico,6,3
...,...,...,...
923,France,1,2
924,Croatia,3,0
925,Morocco,2,0
926,Morocco,2,1


In [29]:
#Concatenar los df
df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby('equipo').mean()
df_team_strength

,Goles_anotados,Goles_recibidos
equipo,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.752941,1.164706
Australia,0.850000,1.850000
Austria,1.576923,1.692308
...,...,...
Uruguay,1.553571,1.303571
Wales,0.625000,1.250000
West Germany,2.050847,1.254237


## Función de predicción de puntos

In [ ]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p

        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)